#### import

In [3]:
import geopandas as gpd
import pandas as pd
import json
import requests
import os
from enum import Enum
from tqdm.notebook import tqdm
import pprint
import shutil
import time
from shapely.geometry import shape
from colorama import Fore, Back, Style

#### missing tools

In [119]:
def missing(df,detail=True):
    total = 0
    for col in df.columns:
        miss = df[col].isnull().sum()
        pct = df[col].isna().mean() * 100
        total += miss
        if miss != 0:
            if pct>10: color=Fore.RED
            else: color=Fore.YELLOW
            print(color+'{} => {} [{}%]'.format(col, miss, round(pct, 2)))
        
        elif (total == 0) and(detail):
            print(Fore.GREEN+'{} => no missing values [{}%]'.format(col, 0))
        total=0

#### Process le dataset Base Nationale des Aménagements Cyclables - Export national OpenStreetMap
"france-********-formatted.geojson"

#### Charge toutes les données

In [1]:
def progress_bar(items):
    start_time = time.time()
    for i, item in enumerate(items):
        bar_length = int(50 * (i+1) / len(items))
        bar = "#" * bar_length + "-" * (50 - bar_length)
        print(f"{i+1}/{len(items)} [{bar}]", end='\r')
        yield item

        
class AccidentData:
    def __init__(self):
        # Répertoire où se trouvent les données
        self.data_dir = os.path.join("..", "data")
        # Nom du fichier JSON contenant les URLs de téléchargement des données
        self.accident_corporels_urls_filename = "accident_corporels_urls.json"
        # Liste des catégories de données disponibles
        self.categories = ["usagers", "vehicules", "lieux", "caracteristiques"]
        # Lecture des URLs à partir du fichier JSON
        self.read_urls()
        # Années pour lesquelles il y a des données disponibles
        self.years = list(self.urls["usagers"].keys())
        # Vérification et contrôle des données
        self.check_and_control_data()
    
    def read_urls(self):
        """
        Charge les URLs des données dans l'attribut self.urls.
        """
        with open(os.path.join(self.data_dir, self.accident_corporels_urls_filename),"r") as file:
            self.urls = json.load(file)
    
    def download_data(self):
        """
        Télécharge les données manquantes.
        """
        # S'il y a des fichiers manquants
        if len(self.filenames) > 0:
            print("[Check] Checking completed, some data is missing!")
            print("[Download] Downloading missing data...")
            # Pour chaque catégorie de données
            for filename in self.filenames.keys():
                filename_dir = os.path.join(self.data_dir,filename) # Répertoire où stocker les données
                print(f"\n[Download] Downloading {filename} files...")
                # Pour chaque année
                for i, year in enumerate(self.filenames[filename]):
                    # Téléchargement des données
                    data = requests.get(self.urls[filename][year]).text

                     # Création du répertoire s'il n'existe pas
                    if not os.path.exists(filename_dir):
                        os.makedirs(filename_dir)

                    # Écriture des données dans un fichier CSV
                    with open(os.path.join(filename_dir,f"{year}.csv"), 'w', encoding='utf-8') as f:
                        f.write(data)

                    # Affichage de la barre de progression
                    bar_length = int(50 * (i+1) / len(self.filenames[filename]))
                    bar = "#" * bar_length + "-" * (50 - bar_length)
                    print(f"{i+1}/{len(self.filenames[filename])} [{bar}]", end='\r')
            print("\n[Download] Download completed!")
        # S'il n'y a pas de fichiers manquants
        else:
            print("[Check] Checking completed, no data is missing!")

    def check_missing_data(self):
        """
        Vérifie les données manquantes.
        """
        print("[Check] Checking if data is in your computer...")
        # Dictionnaire des fichiers manquants par catégorie
        self.filenames = {}
        # Pour chaque catégorie de données
        for categorie in self.categories:
            # Chemin du répertoire de cette catégorie
            filename_path = os.path.join(self.data_dir,categorie)
            # Si le répertoire de cette catégorie n'existe pas, tous les fichiers de cette catégorie sont manquants
            if not os.path.exists(filename_path):
                self.filenames[categorie] = self.years
            # Si le répertoire existe, vérification des fichiers manquants
            else:
                for year in self.years:
                    # Si le fichier de cette année n'existe pas, il est considéré comme manquant
                    if not os.path.exists(os.path.join(filename_path, f"{year}.csv")):
                        # Ajout de l'année au dictionnaire des fichiers manquants
                        if categorie in self.filenames:
                            self.filenames[categorie].append(year)
                        else:
                            self.filenames[categorie] = [year]
                            
    def check_and_control_data(self):
        """
        Vérifie les données manquantes et les télécharge si nécessaire.
        """
        self.check_missing_data()
        self.download_data()
    
    def reset_db(self):
        """
        Réinitialise les données en supprimant tous les répertoires de données et en vérifiant les données manquantes.
        """
        print("[Reset] Reseting data...")
        # Suppression de tous les répertoires de données
        for categorie in self.categories:
            categorie_path = os.path.join(self.data_dir, categorie)
            if os.path.exists(categorie_path):
                shutil.rmtree(categorie_path, ignore_errors=True) 
        print("[Reset] Data have been deleted")
        # Téléchargement des données
        self.check_and_control_data()
        print("[Reset] Data have been reset")
    
    def merge_df(self, begin, end=None, list_cat="All", save=False, name=None):
        """
        Fusionne les données de différentes catégories.

        Parameters:
        begin (int) : Année de début.
        end (int) : Année de fin.
        list_cat (list) : Liste des catégories à fusionner.
        save (bool) : Si True, enregistre le résultat dans un fichier CSV.
        name (str) : Nom du fichier CSV.

        Returns:
        df_final (pandas.DataFrame) : DataFrame fusionné.
        """
        # Si toutes les catégories sont demandées
        if list_cat == "All":
            list_cat = self.categories
        # Si deux ou plusieurs catégories sont demandées
        if len(list_cat) > 1 and len(list_cat) <= len(self.categories):
            # Pour chaque catégorie
            for cat in progress_bar(list_cat):
                # Si la catégorie n'est pas valide
                if cat not in self.categories:
                    raise ValueError("La catégorie de données demandée n'est pas valide")
                # Récupération du DataFrame de la catégorie
                df = self.get_pd_file_from_year(cat=cat, begin=begin, end=end)
                # Si c'est la première catégorie, le DataFrame final est le DataFrame de la catégorie
                if cat == list_cat[0]:
                    df_final = df
                else:
                    # Fusion du DataFrame avec le DataFrame final
                    df_final = pd.merge(df_final, df, on="Num_Acc")
            # Enregistrement du DataFrame final dans un fichier CSV
            if save:
                df_final.to_csv(name)
            return df_final

    def get_pd_file_from_year(self, cat, begin, end=None, merge=True):
        """
        Récupère un DataFrame pandas à partir de l'année demandée.
        
        Parameters:
        cat (str) : Catégorie de données à récupérer.
        begin (int) : Année de début.
        end (int) : Année de fin (optionnel, par défaut None).
        merge (bool) : Si True, fusionne les DataFrames de chaque année en un seul DataFrame. Si False, renvoie une liste de DataFrames.
        
        Returns:
        DataFrame pandas ou liste de DataFrames
        """
        # Si la catégorie de données demandée est valide
        if cat.lower() in self.categories:
            cat_path = os.path.join(self.data_dir, cat)
            # Si une seule année est demandée
            if end == None:
                # Si l'année demandée existe dans les données
                if str(begin) in self.years:
                    # Chargement du fichier CSV en tant que DataFrame pandas
                    return pd.read_csv(os.path.join(cat_path, f"{begin}.csv"), sep=None, engine='python')
            # Si une plage d'années est demandée
            else:
                if str(begin) in self.years and str(end) in self.years:
                    list_df = []
                    for annee in range(begin,end+1):
                        list_df.append(pd.read_csv(os.path.join(cat_path, f"{str(annee)}.csv"), sep=None, engine='python'))
                    if merge:
                        return pd.concat(list_df)
                    else:
                        return list_df
        else:
            raise ValueError("La catégorie de données demandée n'est pas valide")

In [2]:
acc_data = AccidentData()

NameError: name 'json' is not defined

In [76]:
acc_data.get_pd_file_from_year("usagers",2005,2019)

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh,id_vehicule,secu1,secu2,secu3
0,200500000001,1.0,1,4,1,1.0,11.0,0.0,0,0.0,1976.0,A01,NaN,NaN,NaN,NaN
1,200500000001,1.0,1,3,2,3.0,11.0,0.0,0,0.0,1968.0,B02,NaN,NaN,NaN,NaN
2,200500000001,2.0,2,1,1,0.0,11.0,0.0,0,0.0,1964.0,B02,NaN,NaN,NaN,NaN
3,200500000001,4.0,2,1,1,0.0,31.0,0.0,0,0.0,2004.0,B02,NaN,NaN,NaN,NaN
4,200500000001,5.0,2,1,1,0.0,11.0,0.0,0,0.0,1998.0,B02,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132972,201900058837,1.0,1,4,2,1.0,NaN,-1.0,-1,-1.0,1988.0,C01,137Â 982Â 135,1.0,0.0,-1.0
132973,201900058838,1.0,1,4,1,9.0,NaN,-1.0,-1,-1.0,1998.0,A01,137Â 982Â 132,1.0,0.0,-1.0
132974,201900058839,1.0,1,3,1,0.0,NaN,-1.0,-1,-1.0,1979.0,A01,137Â 982Â 131,2.0,0.0,-1.0
132975,201900058840,1.0,1,4,1,0.0,NaN,-1.0,0,-1.0,1974.0,B01,137Â 982Â 129,1.0,0.0,-1.0


In [6]:
data = acc_data.merge_df(2005,2021,save=True, name="all_merged.csv")
data

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,...,agg,int,atm,col,com,adr,gps,lat,long,dep
0,200500000001,1.0,1,4,1,1.0,11.0,0.0,0,0.0,...,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590
1,200500000001,1.0,1,4,1,1.0,11.0,0.0,0,0.0,...,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590
2,200500000001,1.0,1,3,2,3.0,11.0,0.0,0,0.0,...,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590
3,200500000001,1.0,1,3,2,3.0,11.0,0.0,0,0.0,...,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590
4,200500000001,2.0,2,1,1,0.0,11.0,0.0,0,0.0,...,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4763225,202100056517,1.0,1,3,1,0.0,NaN,0.0,0,-1.0,...,1,2,1.0,6.0,33003,Voie rapide Bassens Ambes,NaN,"44,9542747363","-0,5179211363",33
4763226,202100056518,1.0,1,3,1,3.0,NaN,-1.0,-1,-1.0,...,1,1,3.0,1.0,78423,VOLTA (AVENUE),NaN,"48,7966700000","2,0505000000",78
4763227,202100056518,1.0,1,3,1,3.0,NaN,-1.0,-1,-1.0,...,1,1,3.0,1.0,78423,VOLTA (AVENUE),NaN,"48,7966700000","2,0505000000",78
4763228,202100056518,1.0,1,3,1,3.0,NaN,-1.0,-1,-1.0,...,1,1,3.0,1.0,78423,VOLTA (AVENUE),NaN,"48,7966700000","2,0505000000",78


In [10]:

data.columns
# Num_Acc =  df[df["catv"] == 1]["Num_Acc"]
# velo_only = df[df["Num_Acc"].isin(list(Num_Acc))]

Index(['Num_Acc', 'place', 'catu', 'grav', 'sexe', 'trajet', 'secu', 'locp',
       'actp', 'etatp', 'an_nais', 'num_veh_x', 'id_vehicule_x', 'secu1',
       'secu2', 'secu3', 'senc', 'catv', 'occutc', 'obs', 'obsm', 'choc',
       'manv', 'num_veh_y', 'id_vehicule_y', 'motor', 'catr', 'voie', 'v1',
       'v2', 'circ', 'nbv', 'pr', 'pr1', 'vosp', 'prof', 'plan', 'lartpc',
       'larrout', 'surf', 'infra', 'situ', 'env1', 'vma', 'an', 'mois', 'jour',
       'hrmn', 'lum', 'agg', 'int', 'atm', 'col', 'com', 'adr', 'gps', 'lat',
       'long', 'dep'],
      dtype='object')

In [110]:
usagers.head()

,Num_Acc,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202100000001,201 764,B01,1,1,3,1,2000.0,1,0,9,-1,0,0,-1
1,202100000001,201 765,A01,1,1,1,1,1978.0,1,1,-1,-1,0,0,-1
2,202100000002,201 762,A01,1,1,4,1,1983.0,0,1,-1,-1,0,0,-1
3,202100000002,201 763,B01,1,1,3,1,1993.0,0,1,-1,-1,0,0,-1
4,202100000003,201 761,A01,1,1,1,1,1995.0,1,1,0,-1,0,0,-1


In [111]:
#to drop : id_vehicule, num_vehicule, place, secu1, secu2, secu3
usagers = usagers.drop(columns=['id_vehicule', 'num_veh', 'place', 'secu1', 'secu2', 'secu3'])

In [112]:
vehicules.head()

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc
0,202100000001,201 764,B01,1,1,0,2,1,1,5,NaN
1,202100000001,201 765,A01,1,7,0,9,3,17,1,NaN
2,202100000002,201 762,A01,0,7,2,2,1,1,0,NaN
3,202100000002,201 763,B01,0,7,0,2,1,9,0,NaN
4,202100000003,201 761,A01,1,7,0,1,3,1,1,NaN


In [ ]:
vehicules = vehicules.drop(columns=['id_vehicule', 'num_veh','senc', 'manv', 'motor','occutc'])

In [137]:
df = usagers.merge(vehicules,on=['Num_Acc'])
df.head()

,Num_Acc,catu,grav,sexe,an_nais,trajet,locp,actp,etatp,catv,obs,obsm,choc
0,202100000001,1,3,1,2000.0,1,0,0,-1,1,0,2,1
1,202100000001,1,3,1,2000.0,1,0,0,-1,7,0,9,3
2,202100000001,1,1,1,1978.0,1,0,0,-1,1,0,2,1
3,202100000001,1,1,1,1978.0,1,0,0,-1,7,0,9,3
4,202100000002,1,4,1,1983.0,0,0,0,-1,7,2,2,1


In [80]:
#on ne garde dans le dataframe que les lignes implicant un accident entre un vélo et
#un autre usager ou catv == 1 représente les vélos



In [82]:
velo_only.head()

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc,catu,grav,sexe,an_nais,trajet,locp,actp,etatp
0,202100000001,201 764,B01,1,1,0,2,1,1,5,NaN,1,3,1,2000.0,1,0,0,-1
1,202100000001,201 764,B01,1,1,0,2,1,1,5,NaN,1,1,1,1978.0,1,0,0,-1
2,202100000001,201 765,A01,1,7,0,9,3,17,1,NaN,1,3,1,2000.0,1,0,0,-1
3,202100000001,201 765,A01,1,7,0,9,3,17,1,NaN,1,1,1,1978.0,1,0,0,-1
18,202100000006,201 752,B01,2,1,0,0,5,1,5,NaN,1,4,1,2009.0,2,0,0,-1


In [51]:
usagers_velo.describe()

,Num_Acc,catu,grav,sexe,an_nais,trajet,locp,etatp
count,1.065100e+04,10651.000000,10651.000000,10651.000000,10255.000000,10651.000000,10651.000000,10651.000000
mean,2.021000e+11,1.092667,2.451882,1.191719,1978.862311,3.356305,-0.375645,-0.929678
std,1.535996e+04,0.381440,1.394021,0.614879,19.424436,2.862299,0.890958,0.410236
min,2.021000e+11,1.000000,-1.000000,-1.000000,1924.000000,-1.000000,-1.000000,-1.000000
25%,2.021000e+11,1.000000,1.000000,1.000000,1964.000000,1.000000,-1.000000,-1.000000
50%,2.021000e+11,1.000000,3.000000,1.000000,1981.000000,4.000000,0.000000,-1.000000
75%,2.021000e+11,1.000000,4.000000,2.000000,1995.000000,5.000000,0.000000,-1.000000
max,2.021001e+11,3.000000,4.000000,2.000000,2021.000000,9.000000,9.000000,3.000000
